In [89]:
import pandas as pd
import numpy as np
import time

### Exploration of the data

In [17]:
base_path = 'C:\\Users\\nicor\\OneDrive\\Documentos\\Python\\Kaggle\\H&M Personalize Fashion Recommender system - datasets\\'

In [18]:
#### loading csv files into dataframes

In [15]:
#load articles dataset
articles = pd.read_csv(base_path + 'articles.csv')

#load customers dataset
customers = pd.read_csv(base_path + 'customers.csv')

#load transactions dataset
transactions = pd.read_csv(base_path + 'transactions_train.csv')

#load sample submission dataset
sample_submission = pd.read_csv(base_path + 'sample_submission.csv')

In [76]:
#Exploration of the articles dataset
articles.info()
articles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [77]:
#filtering articles dataset
articles_filtered = articles.drop(columns=['graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name'])


In [66]:
#Exploration of na values in articles dataset
articles_filtered.isnull().sum() #no na values! 

article_id                    0
product_code                  0
product_type_no               0
product_group_name            0
department_no                 0
perceived_colour_master_id    0
index_code                    0
index_group_no                0
section_no                    0
garment_group_no              0
dtype: int64

In [33]:
#Exploration of the customers dataset
customers.info()
customers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [48]:
#filtering columns of customers dataset
customers_filtered = customers[['customer_id', 'Active', 'age']]

In [64]:
total_entries_c = len(customers_filtered)

#Exploration of na values in customer_id column
print(customers_filtered.customer_id.isna().sum()) #no na values

#Exploration of na values in Active column
print(f'The amount of NA values in the Active col is:{customers_filtered.Active.isna().sum()} ({round(customers_filtered.Active.isna().sum()/total_entries_c*100,2)}%)') 

#Replacing na values in Active column with 0
customers_filtered.Active.fillna(0, inplace=True)

#Exploration of na values in age column
print(f'The amount of NA values in the Active col is:{customers_filtered.age.isna().sum()} ({round(customers_filtered.age.isna().sum()/total_entries_c*100,2)}%)') 

#Replacing na values in age column with the average of the age column
customers_filtered.age.fillna(customers_filtered.age.mean(), inplace=True)

0
The amount of NA values in the Active col is:907576 (66.15%)
The amount of NA values in the Active col is:15861 (1.16%)
(1371980, 3)
(1371980, 7)


C:\Users\nicor\anaconda3\envs\mne3\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [85]:
customers_filtered.age.values.reshape(-1,1)

array([[49.],
       [25.],
       [24.],
       ...,
       [21.],
       [18.],
       [65.]])

In [101]:
#Clustering of the age column
#customers_filtered.age.hist(bins=50)

#k means clustering
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

label = KMeans(n_clusters=5, random_state=0).fit_predict(customers_filtered.age.values.reshape(-1,1))

#plot_age_cluster = pd.DataFrame(label.cluster_centers_, columns=['age_cluster'])
#plot_age_cluster['age_cluster'] = plot_age_cluster['age_cluster'].astype(int).sort_values().reset_index(drop=True)

print(label)

filtered_label0 = customers_filtered.age.values.reshape(-1,1)[label == 0]


print(filtered_label0)

plt.scatter(filtered_label0[:,0] , filtered_label0[:,1])
plt.show()

#Getting unique labels
u_labels = np.unique(label)

#plotting the results:
 
#for i in u_labels:
 #   plt.scatter(customers_filtered.age[label == i , 0] , customers_filtered.age[label == i , 1] , label = i)
plt.legend()
plt.show()



[2 1 1 ... 1 1 4]
[[36.38696457]
 [41.        ]
 [35.        ]
 ...
 [36.        ]
 [40.        ]
 [40.        ]]


IndexError: index 1 is out of bounds for axis 1 with size 1

In [100]:
filtered_label0

array([[36.38696457],
       [41.        ],
       [35.        ],
       ...,
       [36.        ],
       [40.        ],
       [40.        ]])

In [67]:
#Exploration of transaction dataset
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [71]:
#Exploration of na values in transaction dataset
total_entries_t = len(transactions)

for item in transactions.columns: #no na values! 
    print(f'The amount of NA values in the {item} column is:{transactions[item].isna().sum()} ({round(transactions[item].isna().sum()/total_entries_t*100,2)}%)')

The amount of NA values in the t_dat column is:0 (0.0%)
The amount of NA values in the customer_id column is:0 (0.0%)
The amount of NA values in the article_id column is:0 (0.0%)
The amount of NA values in the price column is:0 (0.0%)
The amount of NA values in the sales_channel_id column is:0 (0.0%)


In [ ]:
#Creating input 